## ETL -4

In [1]:
import pandas as pd
import mysql.connector

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras.
📌 Nota Todo lo tendremos que hacer desde jupyter notebook

1. Cread la BBDD con el nombre de energía. La BBDD tendrá la siguiente forma:


In [2]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab", # aquí tendréis que poner vuestra contraseña de MySQL
      auth_plugin = 'mysql_native_password') # esta línea no es necesaria solo lo tendréis que incluir si os sale el siguiente error: Authentication plugin 'caching_sha2_password' is not supported
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [3]:
crear_bbdd('energía')

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS energía;


2. Cread las tablas de la BBDD:

Tabla fechas

Tabla nacional_renovable_no_renovable

Tabla comunidades_renovable_no_renovable

Tabla comunidades

Aspecto que deberán tener las tablas:

![tablas](https://github.com/Adalab/data_imagenes/blob/main/Modulo-2/Limpieza/esquema_BBDD.png?raw=true)

In [4]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}",  
                                     auth_plugin = 'mysql_native_password')
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [5]:
tabla_comunidades = '''
CREATE TABLE IF NOT EXISTS `energía`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidad` VARCHAR(45),
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
'''

In [36]:
crear_insertar_tabla('energía', 'AlumnaAdalab', tabla_comunidades)

In [7]:
tabla_nacional_renovable_no_renovable = """
CREATE TABLE IF NOT EXISTS `energía`.`nacional_renovable_no_renovable`(
`idnacional` INT NOT NULL AUTO_INCREMENT,
`porcentaje` DECIMAL,
`tipo_energia` VARCHAR(45),
`valor` DECIMAL,
`comunidades_idcomunidades` INT,
PRIMARY KEY (`idnacional`))
ENGINE = InnoDB;"""

In [8]:
crear_insertar_tabla('energía', 'AlumnaAdalab', tabla_nacional_renovable_no_renovable)

In [14]:
tabla_comunidades_renovable_no_renovable = """
    CREATE TABLE IF NOT EXISTS `energía`.`comunidades_renovable_no_renovable`(
    `id` INT NOT NULL AUTO_INCREMENT,
    `porcentaje` DECIMAL,
    `tipo_energia` VARCHAR(45),
    `valor` DECIMAL,
    `idcomunidades` INT NOT NULL,
    PRIMARY KEY (`id`),
    CONSTRAINT `fk_comunidades_comunidades`
        FOREIGN KEY (`idcomunidades`)
        REFERENCES `energía`.`comunidades` (`idcomunidades`))
    ENGINE = InnoDB;"""

In [15]:
crear_insertar_tabla('energía', 'AlumnaAdalab', tabla_comunidades_renovable_no_renovable)

In [16]:
tabla_fechas = """
    CREATE TABLE IF NOT EXISTS `energía`.`fechas`(
    `idFechas` INT NOT NULL AUTO_INCREMENT,
    `fecha` DATE,
    `nacional_renovable_no_renovable_id` INT,
    `comunidades_renovable_no_renovable_id` INT,
    PRIMARY KEY (`idFechas`),
    CONSTRAINT `fk_nacional_fechas`
        FOREIGN KEY(`nacional_renovable_no_renovable_id`)
        REFERENCES `energía`.`nacional_renovable_no_renovable`(`idnacional`),
    CONSTRAINT `fk_comunidades_fechas`
        FOREIGN KEY (`comunidades_renovable_no_renovable_id`)
        REFERENCES `energía`.`comunidades_renovable_no_renovable` (`id`))
    ENGINE = InnoDB;"""

In [17]:
crear_insertar_tabla('energía', 'AlumnaAdalab', tabla_fechas)

In [29]:
df_nacional = pd.read_csv('../datos/df_energia_nacional.csv', index_col = 0)
df_ccaa = pd.read_csv('../datos/df_ccaa_clean.csv', index_col = 0)
df_comunidades = pd.read_csv('../datos/poblacion_comunidades_clean.csv', index_col = 0)

In [26]:
df_comunidades.sample(3)

,Comunidad,indice,Año,Total
145,Castilla - La Mancha,7,2002,1.782.038
406,Canarias,8742,2016,2.101.924
14,Ceuta,8744,2008,77.389


In [24]:
df_ccaa.sample(3)

,value,percentage,type,CCAA,fecha
261,15191368.30,0.81,no renovable,Comunitat Valenciana,2019-01-01
218,20594544.96,0.77,renovable,Castilla y León,2018-01-01
275,1248386.13,0.77,no renovable,Cantabria,2020-01-01


In [27]:
df_nacional.sample(3)

,value,percentage,datetime,type
12,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,no renovable
13,1.977799e+08,0.698576,2012-01-01T00:00:00.000+01:00,no renovable
20,1.629155e+08,0.624607,2019-01-01T00:00:00.000+01:00,no renovable


BONUS 

3. Insertar los datos en las tablas.

📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [33]:
def check_comunidades(contraseña, nombre_bbdd):
    
    cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
    mycursor = cnx.cursor()


    query_existe_comunidad = f"""
            SELECT DISTINCT comunidad FROM comunidades
            """
    mycursor.execute(query_existe_comunidad)
    comunidades = mycursor.fetchall()
    return comunidades

In [35]:
check_comunidades("AlumnaAdalab", "energía")

[('La Rioja',),
 ('Ceuta',),
 ('Melilla',),
 ('Andalucía',),
 ('Aragón',),
 ('Cantabria',),
 ('Castilla - La Mancha',),
 ('Castilla y León',),
 ('Cataluña',),
 ('País Vasco',),
 ('Principado de Asturias',),
 ('Comunidad de Madrid',),
 ('Comunidad Foral de Navarra',),
 ('Comunitat Valenciana',),
 ('Extremadura',),
 ('Galicia',),
 ('Illes Balears',),
 ('Canarias',),
 ('Región de Murcia',)]

In [38]:
for indice, fila in df_comunidades.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_comunidades = f"""
                INSERT INTO comunidades (comunidad) 
                VALUES ( "{fila["Comunidad"]}");
                """  
    
    comunidades = check_comunidades("AlumnaAdalab", "energía")
    
    if len(comunidades) == 0 or fila['Comunidad'] not in comunidades[0]: 
        crear_insertar_tabla('energía', 'AlumnaAdalab', query_comunidades)

    else:
        print(f"{fila['Comunidad']} ya esta en nuestra BBDD") 
     

La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
La Rioja ya esta en nuestra BBDD
